In [1]:
import pickle # 리스트 안의 데이터프레임 형태 저장
import requests # api 요청
import json
import pandas as pd
import numpy as np
import time

In [2]:
#https://developer.riotgames.com/에서 회원가입 후 api key 만들어주세요.
#시작 인덱스, 끝 인덱스 지정
init = 0 
end = 60000

In [3]:
#데이터 읽어오기 - 사용자 이름 저장
league_df = pd.read_csv('LOL_User_IDs.csv', index_col=0)
league_df.reset_index(inplace=True)

In [4]:
# 데이터 읽어오기 - 매치 데이터 저장
match_info_df = pd.read_csv('MatchInfoData.csv', index_col=0)
match_info_df.reset_index(inplace=True)

In [4]:
league_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3331 entries, 0 to 3330
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         3331 non-null   int64 
 1   tier          3331 non-null   object
 2   summonerId    3331 non-null   object
 3   summonerName  3331 non-null   object
 4   leaguePoints  3331 non-null   int64 
 5   wins          3331 non-null   int64 
 6   losses        3331 non-null   int64 
 7   veteran       3331 non-null   bool  
 8   inactive      3331 non-null   bool  
 9   freshBlood    3331 non-null   bool  
 10  hotStreak     3331 non-null   bool  
 11  account_id    3331 non-null   object
dtypes: bool(4), int64(4), object(4)
memory usage: 221.3+ KB


In [6]:
match_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333051 entries, 0 to 333050
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   index       333051 non-null  int64 
 1   platformId  333051 non-null  object
 2   gameId      333051 non-null  int64 
 3   champion    333051 non-null  int64 
 4   queue       333051 non-null  int64 
 5   season      333051 non-null  int64 
 6   timestamp   333051 non-null  int64 
 7   role        333051 non-null  object
 8   lane        333051 non-null  object
dtypes: int64(6), object(3)
memory usage: 22.9+ MB


In [7]:
match_gameid = match_info_df.gameId.drop_duplicates() #gameId 리스트로만. 중복 제거.

In [8]:
match_gameid.index = range(len(match_gameid)) #이후 인덱스도 맞추기

In [ ]:
#Match 정보 분산.
match_df = pd.DataFrame()
for game_id in match_gameid[init, end]: # 이전의 매치에 대한 정보 데이터에서 게임 아이디를 가져온다
    api_url = 'https://kr.api.riotgames.com/lol/match/v4/matches/' + str(game_id) + '?api_key=' + api_key
    r = requests.get(api_url)
    while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 5초 간 시간을 지연
        time.sleep(5)
        r = requests.get(api_url)
    r_json = r.json()
    temp_df = pd.DataFrame(list(r_json.values()), index=list(r_json.keys())).T # 게임 아이디에 대한 매치 데이터를 받아서 추가
    trueset = [ #게임 가져올 조건 지정.
               temp_df.gameMode[0] == 'CLASSIC', #클래식일 때만
               temp_df.gameVersion[0][0:5] in ['10.20', '10.21', '10.22'], #최신 패치만 가져오기
               temp_df.gameType[0] == 'MATCHED_GAME', #gametype이 Matched-game일때만
               temp_df.gameDuration[0]>=300, #게임 시간 5분 이상
               temp_df.queueId[0] in [420, 430, 440]
              ] 
    if all(trueset): #조건 만족할 때만 가져오기
        temp_df = temp_df.drop(columns=['platformId', 'mapId', 'seasonId', 'gameMode', 'gameType' ], axis=1) 
        #우선 불필요한 정보는 드롭하고... 
        match_df = pd.concat([match_df, temp_df])

In [ ]:
#csv 파일 대신 피클 형태로 저장해보자.
match_df.to_pickle(f'Gameinfo_{init}_to_{end}.pickle')